In [ ]:
from skyfield.api import load, EarthSatellite
import pandas as pd
import numpy as np
from itertools import combinations

# 🚀 Step 1: Load TLE Data
df = pd.read_csv("tle_data.csv")

# 🚀 Step 2: Load Time Scale
ts = load.timescale()

# 🚀 Step 3: Generate Satellite Objects
satellites = {}
for _, row in df.iterrows():
    try:
        satellite = EarthSatellite(row["TLE Line 1"], row["TLE Line 2"], row["Satellite Name"], ts)
        satellites[row["Satellite Name"]] = satellite
    except Exception as e:
        print(f"⚠️ Error processing {row['Satellite Name']}: {e}")

# 🚀 Step 4: Define Collision Threshold (e.g., 10 km)
collision_threshold_km = 10.0

# 🚀 Step 5: Compute Distances Between Satellite Pairs
collision_warnings = []
time_now = ts.now()

for sat1, sat2 in combinations(satellites.keys(), 2):  # Compare all satellite pairs
    pos1 = satellites[sat1].at(time_now).position.km
    pos2 = satellites[sat2].at(time_now).position.km
    distance = np.linalg.norm(np.array(pos1) - np.array(pos2))  # Euclidean Distance
    
    if distance < collision_threshold_km:
        collision_warnings.append((sat1, sat2, distance))

# 🚀 Step 6: Print Possible Collisions
if collision_warnings:
    print("\n⚠️ Potential Collisions Detected!")
    for sat1, sat2, dist in collision_warnings:
        print(f"🚨 {sat1} & {sat2} may collide! Distance: {dist:.2f} km")
else:
    print("✅ No imminent collisions detected.")
